# Ollama server

In [2]:
%pip -q install llama-index llama-index-readers-web llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-readers-file

In [3]:
! curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10406    0 10406    0     0  40309      0 --:--:-- --:--:-- --:--:-- 40333
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])


In [5]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [7]:
!ollama pull llama3

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling ma

In [8]:
!ollama list

NAME         	ID          	SIZE  	MODIFIED      
llama3:latest	365c0bd3c000	4.7 GB	4 seconds ago	


# Libraries

In [9]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.readers.web import SimpleWebPageReader
from llama_index.readers.file import (
    DocxReader,
    HWPReader,
    PDFReader,
    EpubReader,
    FlatReader,
    HTMLTagReader,
    ImageCaptionReader,
    ImageReader,
    ImageVisionLLMReader,
    IPYNBReader,
    MarkdownReader,
    MboxReader,
    PptxReader,
    PandasCSVReader,
    VideoAudioReader,
    UnstructuredReader,
    PyMuPDFReader,
    ImageTabularChartReader,
    XMLReader,
    PagedCSVReader,
    CSVReader,
    RTFReader,
)
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)


# File to url Mapping

In [51]:
file_to_url_mapping={}

In [52]:
def table_to_csv_converter(url,file_to_url_mapping):

  from bs4 import BeautifulSoup
  import requests
  import pandas as pd
  import os

  url = url
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html')

  # Detect all tables on the webpage
  all_tables = soup.find_all('table')
  print(f"Total number of tables: {len(all_tables)}")

  # Create a directory to save the CSV files
  directory = 'directory_csv'
  if not os.path.exists(directory):
      os.makedirs(directory)

  # Iterate over each table
  for i, table in enumerate(all_tables):
      # Get the caption of the table
      table_caption = table.find('caption').text.strip()

      world_titles = table.find_all('th')
      world_table_titles = [title.text.strip() for title in world_titles]

      df = pd.DataFrame(columns=world_table_titles)

      column_data = table.find_all('tr')
      for row in column_data[1:]:
          row_data = row.find_all('td')
          individual_row_data = [data.text.strip() for data in row_data]
          df.loc[len(df)] = individual_row_data

      # Save the DataFrame as a CSV file with the table caption as the file name
      csv_file = os.path.join(directory, f'{table_caption}.csv')
      df.to_csv(csv_file, index=False)
      print(f"CSV file saved: {csv_file}")
      #file_to_url_mapping.append({ f'{table_caption}.csv':url})
      file_to_url_mapping[f'{table_caption}.csv'] = url

In [53]:
url_with_table = ['https://maharashtra.nic.in/directory/','https://maharashtra.nic.in/rti/','https://maharashtra.nic.in/district-centres/']

In [54]:
for url in url_with_table:
  table_to_csv_converter(url,file_to_url_mapping)

Total number of tables: 37
CSV file saved: directory_csv/SIO.csv
CSV file saved: directory_csv/Maharashtra State Centre, Mumbai.csv
CSV file saved: directory_csv/Ahmadnagar.csv
CSV file saved: directory_csv/Akola.csv
CSV file saved: directory_csv/Amravati.csv
CSV file saved: directory_csv/Aurangabad.csv
CSV file saved: directory_csv/Beed.csv
CSV file saved: directory_csv/Bhandara.csv
CSV file saved: directory_csv/Buldana.csv
CSV file saved: directory_csv/Chandrapur.csv
CSV file saved: directory_csv/Dhule.csv
CSV file saved: directory_csv/Gadchiroli.csv
CSV file saved: directory_csv/Gondia.csv
CSV file saved: directory_csv/Hingoli.csv
CSV file saved: directory_csv/Jalgaon.csv
CSV file saved: directory_csv/Jalna.csv
CSV file saved: directory_csv/Kolhapur.csv
CSV file saved: directory_csv/Mumbai.csv
CSV file saved: directory_csv/Mumbai Suburban (Bandra).csv
CSV file saved: directory_csv/Nagpur.csv
CSV file saved: directory_csv/Nanded.csv
CSV file saved: directory_csv/Nandurbar.csv
CSV fil

In [55]:
file_to_url_mapping

{'SIO.csv': 'https://maharashtra.nic.in/directory/',
 'Maharashtra State Centre, Mumbai.csv': 'https://maharashtra.nic.in/directory/',
 'Ahmadnagar.csv': 'https://maharashtra.nic.in/directory/',
 'Akola.csv': 'https://maharashtra.nic.in/directory/',
 'Amravati.csv': 'https://maharashtra.nic.in/directory/',
 'Aurangabad.csv': 'https://maharashtra.nic.in/directory/',
 'Beed.csv': 'https://maharashtra.nic.in/directory/',
 'Bhandara.csv': 'https://maharashtra.nic.in/directory/',
 'Buldana.csv': 'https://maharashtra.nic.in/directory/',
 'Chandrapur.csv': 'https://maharashtra.nic.in/directory/',
 'Dhule.csv': 'https://maharashtra.nic.in/directory/',
 'Gadchiroli.csv': 'https://maharashtra.nic.in/directory/',
 'Gondia.csv': 'https://maharashtra.nic.in/directory/',
 'Hingoli.csv': 'https://maharashtra.nic.in/directory/',
 'Jalgaon.csv': 'https://maharashtra.nic.in/directory/',
 'Jalna.csv': 'https://maharashtra.nic.in/directory/',
 'Kolhapur.csv': 'https://maharashtra.nic.in/directory/',
 'Mum

# Model and Embeddings

In [38]:
# Initialize settings for LlamaIndex
Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)

In [11]:

Settings.embed_model = HuggingFaceEmbedding(model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

# Loading the index

In [39]:
persist_dir = "/content/combine_index"

storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context)

print(index)

# Chat_engine

In [40]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
loaded_chat_store = SimpleChatStore.from_persist_path(
    persist_path="chat_store.json"
)

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=loaded_chat_store,
    chat_store_key="user1",
)


chat_engine = index.as_chat_engine(
    chat_mode="context",
    system_prompt="You are a helpful AI assistant. You are expert in retrieving the answer for the user input based on the provided context. You are also able to give the source link(url) to each user input.",
    memory=chat_memory, verbose = True
)



In [62]:
# response = chat_engine.chat("what is the published date of Implementation of Aadhar Enabled Biometric Attendance System (AEBAS) in NIC Maharashtra?")

query_engine = index.as_query_engine()

query = "what are NIC services?"
val_query_response = query_engine.query(query)
# print(f"RESPONSE:\n{response}")
# print(response)

In [63]:
val_query_response

Response(response="NIC provides different kinds of E-mail services to its users, over NICNET, which is NIC's satellite-based communication network. The different types of e-mail services being provided include SMTP, UUCP and X.400.", source_nodes=[NodeWithScore(node=TextNode(id_='d0cb2e6f-e209-41e4-9501-e7d080f3cc86', embedding=None, metadata={'URL': 'https://servicedesk.nic.in/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://servicedesk.nic.in/', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'URL': 'https://servicedesk.nic.in/'}, hash='225fd4a23a5f7ca562a54fb6e6a9f08987431b4d399c586ac6e0c374bffcfd49')}, text='Home | NIC Service Desk | Ministry of Electronics and IT | Department of NIC | GoI', start_char_idx=0, end_char_idx=82, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7082680454313587), NodeWithScore(node=TextNode

In [57]:
# response = chat_engine.chat("what is the published date of Implementation of Aadhar Enabled Biometric Attendance System (AEBAS) in NIC Maharashtra?")

query_engine = index.as_query_engine()

query = "who isz ther RTI Public Info Officer?"
val_query_response = query_engine.query(query)
# print(f"RESPONSE:\n{response}")
# print(response)

In [58]:
val_query_response

Response(response='Smt. Ireni Akoijam is the RTI Public Information Officer.', source_nodes=[NodeWithScore(node=TextNode(id_='de440ef6-2ec9-4dfc-8e2c-5d2fbb4022fb', embedding=None, metadata={'file_path': '/content/RTI_csv/PUBLIC INFORMATION OFFICERS ( PIO ).csv', 'file_name': 'PUBLIC INFORMATION OFFICERS ( PIO ).csv', 'file_type': 'text/csv', 'file_size': 240, 'creation_date': '2024-05-24', 'last_modified_date': '2024-05-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='896ad005-3f53-4edd-8b2c-9f5102715735', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/RTI_csv/PUBLIC INFORMATION OFFICERS ( PIO ).csv', 'file_name': 'PUBLIC INFORMATION OFFICERS ( PIO ).csv', 'file_type': 'text/cs

In [64]:

#print("Jigyasa Response: " + str(val_query_response.response))
node = val_query_response.source_nodes[0]
response_json = {}
response_json['response'] = val_query_response.response
response_json['search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])







In [65]:
response_json

{'response': "NIC provides different kinds of E-mail services to its users, over NICNET, which is NIC's satellite-based communication network. The different types of e-mail services being provided include SMTP, UUCP and X.400.",
 'search_Score': 0.7082680454313587,
 'url': 'https://servicedesk.nic.in/'}

# Chat_store

In [ ]:
chat_store.persist(persist_path="chat_store.json")
loaded_chat_store = SimpleChatStore.from_persist_path(
    persist_path="chat_store.json"
)